# 7장 데이터 준비하기: 다듬기, 변형, 병합
## 데이터 합치기

- pandas.merge는 하나 이상의 키를 기준으로 로우를 병합 (i.e. *join* in SQL)
- pandas.concat은 하나의 축을 따라 객체를 이어붙임
- combine_first 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체의 값으로 채움

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

___
## 1. 데이터베이스 스타일로 DataFrame 합치기

In [2]:
df1 = DataFrame({'key': list('bbacaab'),
                'data1': range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [3]:
df2 = DataFrame({'key': list('abd'),
               'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [4]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


- merge 함수는 겹치는 칼럽의 이름을 키로 사용

In [5]:
pd.merge(df1, df2, on = 'key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


- 가능한 명시적으로 on = '' 지정하는 것이 바람직

In [6]:
df3 = DataFrame({'lkey': list('bbacaab'),
                'data1': range(7)})
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [7]:
df4 = DataFrame({'rkey': list('abd'),
                'data2': range(3)})
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [8]:
pd.merge(df3, df4, left_on= 'lkey', right_on = 'rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


- 공통 칼럼이 없을 시 따로 지정해줄 필요가 있음
- key에 'c' 값이 없는 것은 기본적으로 merge 함수의 default 값이 inner join이기 때문

In [9]:
pd.merge(df1, df2, how = 'outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


- 이상 1대 다 병합 (병합하는 컬럼 중 unique 값을 가지고 있는 DataFrame이 있음)

___

In [10]:
df1 = DataFrame({'key': list('bbacab'),
                'data1': range(6)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [11]:
df2 = DataFrame({'key': list('ababd'),
                'data2': range(5)})
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [12]:
pd.merge(df1, df2, on = 'key', how = 'left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


- 다대 다 조인은 두 로우의 데카르트 곱을 반환 (df1의 b 3개, df2의 b 2개의 곱으로 총 6개의 'b' 로우 반환)

In [13]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                 'key2': ['one', 'two', 'one'],
                 'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                  'key2': ['one', 'one', 'one', 'two'],
                  'rval': [4, 5, 6, 7]})

In [14]:
pd.merge(left, right, on = ['key1', 'key2'], how= 'outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


- 2개 이상의 key 값으로 merge 시, on = [칼럼1, 칼럼2] 로 merge 가능

In [15]:
pd.merge(left, right, on = 'key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [16]:
pd.merge(left, right, on= 'key1', suffixes=['_l', '_r'])

,key1,key2_l,lval,key2_r,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


- 칼럼 명 중첩 시 '_x', '_y'가 따라 붙으며, suffixes =  옵션으로 지정 가능
___
## 2. 색인 머지하기

In [17]:
left1 = DataFrame({'key': list('abaabc'),
                  'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [18]:
right1 = DataFrame({'group_val': [3.5, 7]}, index = ['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [19]:
pd.merge(left1, right1, left_on = 'key', right_index= True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


- index를 on 값으로 대체해서 merge 가능

In [20]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                  'key2': [2000, 2001, 2002, 2001, 2002],
                  'data': np.arange(5.)})
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [21]:
righth = DataFrame(np.arange(12).reshape((6,2)),
                   index = [['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                           [2001,2000, 2000, 2000, 2001, 2002]],
                   columns = ['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [22]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


- 계층 색인된 데이터도 마찬가지로 merge 처리

In [23]:
left2 = DataFrame([[1,2,], [3,4], [5,6]],
                   index =  list('ace'),
                 columns = ['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [24]:
right2 = DataFrame([[7,8], [9,10], [11,12], [13,14]],
                  index = list('bcde'),
                  columns = ['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [25]:
pd.merge(left2, right2, how = 'outer', left_index = True, right_index = True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


- 양쪽 다 색인만 사용해서 merge 가능

In [26]:
left2.join(right2, how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


- 색인만으로 merge할 경우 DataFrame의 merge 메서드 사용하면 훨씬 간편

In [27]:
left1.join(right1, on = 'key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


- Default는 left_join임

In [28]:
another = DataFrame([[7,8], [9,10], [11,12], [16,17]],
                   index = list('acef'),
                   columns = ['New York', 'Oregon'])
another

,New York,Oregon
a,7,8
c,9,10
e,11,12
f,16,17


In [29]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
c,3,4,9.0,10.0,9,10
e,5,6,13.0,14.0,11,12


In [30]:
left2.join([right2, another], how = 'outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


- 색인대 색인으로 두 DataFrame을 합치려면 DataFrame 리스트를 join 메서드에 넘기면 됨
- 하지만 이런 경우 일반적으로 아래의 concat 메서드를 사용
___
## 3. 축 따라 이어붙이기

In [32]:
arr = np.arange(12).reshape(3,4)
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [33]:
np.concatenate([arr,arr], axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

- np.concatenate로 일반적인 축 병합 가능
- pd.concat으로 다양한 사황들 고려 (색인이 상이, 어떤 축으로 연결, 합쳐지기 전과 후 구분)

In [34]:
s1 = Series([0,1], index = ['a','b'])
s2 = Series([2,3,4], index = ['c', 'd', 'e'])
s3 = Series([5, 6], index = ['f', 'g'])

In [35]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [36]:
pd.concat([s1, s2, s3], axis = 1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [42]:
s4 = pd.concat([s1*5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [39]:
pd.concat([s1, s4], axis = 1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [41]:
pd.concat([s1, s4], axis = 1, join = 'inner')

,0,1
a,0,0
b,1,5


In [46]:
pd.merge(DataFrame(s1), DataFrame(s4), left_index= True, right_index=True)

,0_x,0_y
a,0,0
b,1,5


- join = 'inner'로 교집합 구하는 것도 가능 (merge 와 유사한 기능)

In [48]:
pd.concat([s1,s4], axis = 1, join_axes = [['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


- join_axes = [[   ]]로 merge 하려는 행 지정 가능

In [49]:
result = pd.concat([s1, s1, s3], keys = ['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [52]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


___

In [54]:
df1 = DataFrame(np.arange(6).reshape(3,2),
               index = list('abc'),
               columns = ['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [57]:
df2 = DataFrame(5 + np.arange(4).reshape(2,2),
               index = ['a', 'c'],
               columns = ['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [62]:
pd.concat([df1, df2], axis = 1, keys = ['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [65]:
pd.concat({'level1': df1, 'level2': df2}, axis =1, names = ['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

- 리스트 대신 사전으로도 concat 가능하며, 이때 사전의 키가 keys 옵션으로 사용됨

In [66]:
df1 = DataFrame(np.random.randn(3,4), columns = list('abcd'))
df1

,a,b,c,d
0,0.874046,0.135443,0.649309,-0.631571
1,0.040930,0.384826,0.751149,-1.126025
2,-0.510751,1.362888,1.132175,-0.275254


In [67]:
df2 = DataFrame(np.random.randn(2,3), columns = list('bda'))
df2

,b,d,a
0,0.516396,0.112072,-0.111806
1,0.794781,-0.167698,-0.267154


In [68]:
pd.concat([df1, df2])

,a,b,c,d
0,0.874046,0.135443,0.649309,-0.631571
1,0.040930,0.384826,0.751149,-1.126025
2,-0.510751,1.362888,1.132175,-0.275254
0,-0.111806,0.516396,NaN,0.112072
1,-0.267154,0.794781,NaN,-0.167698


In [69]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.874046,0.135443,0.649309,-0.631571
1,0.040930,0.384826,0.751149,-1.126025
2,-0.510751,1.362888,1.132175,-0.275254
3,-0.111806,0.516396,NaN,0.112072
4,-0.267154,0.794781,NaN,-0.167698


- 로우 색인이 불필요한 경우 ignore_index 명령어를 사용
___
## 4. 겹치는 데이터 합치기

In [70]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
          index = list('fedcba'))
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [72]:
b = Series(np.arange(len(a), dtype = np.float64),
          index = list('fedcba'))
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [73]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

- ** np.where은 벡터화된 if-else문을 구문으로 표현하는 함수**

In [75]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

- Series 객체의 combine_fist 메서드는 누락된 데이터를 새로운 객체의 값으로 채워 넣음

In [78]:
df1 = DataFrame({'a': [1, np.nan, 5, np.nan],
               'b': [np.nan, 2, np.nan, 6],
               'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [79]:
df2 = DataFrame({'a': [5, 4, np.nan, 3, 7],
                'b': [np.nan, 3, 4, 6, 8]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [80]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


- combine_first 메서드는 데이터프레임에서도 사용 가능